In [247]:
!pip install -U langchain-ollama langchain-openai

In [248]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [249]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [250]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

In [251]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()
df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [252]:
df.shape

(40000, 2)

In [253]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [254]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [255]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [256]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [257]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [258]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.04 s, sys: 13.7 ms, total: 2.05 s
Wall time: 3.93 s


In [259]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [260]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 5.16 s, sys: 33.6 ms, total: 5.19 s
Wall time: 5.38 s


In [261]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [262]:
coffee_reviews_df.shape

(4981, 2)

In [263]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [264]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [265]:
!ollama pull gemma3:270m

In [266]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
gemma3:270m                           e7d36fb2c3b3    291 MB    Less than a second ago    
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    9 minutes ago             


In [267]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊




In [268]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T08:16:18.93826071Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T08:06:27.091569248Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [269]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [270]:
llm = OllamaLLM(model="gemma3:270m")

In [271]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ 😊


In [272]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [273]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [274]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [275]:
chain = prompt | llm

In [276]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [277]:
response

'```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  sum = 0\n  for number in numbers:\n    sum += number\n  return sum\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1. **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะใช้ในการเรียกใช้งานโค้ด\n2. **`sum = 0`**:  สร้างค่าคงสองตัว (`sum`)  เพื่อเก็บผลรวม\n3. **`for number in numbers:`**:  วนซ้ำผ่านทุกตัวเลขใน `numbers`  (แต่ละตัวเลขจะถูกนำไปเก็บเป็นค่าใน `sum`)\n4. **`sum += number`**:  เพิ่มค่า `number` เข้าไปใน `sum`  (ค่าที่เก็บผลรวม)\n5. **`return sum`**:  คืนค่า `sum`  (ผลรวมของตัวเลขทั้งหมด)\n6. **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวอย่าง `numbers`  ซึ่งมี 10 ตัวเลข\n7. **`result = find_sum(numbers)`**:  เรียกใช้ `find_sum`  เพื่อหาผลรวมของ `numbers`\n8. **`print(result)`**:  แสดงผลลัพธ์ที่ได้จาก `find_sum`\n**ข้อดีของโค้ดนี้:**\n\n* **อ่านง่าย:**  โค้ดนี้ใช้ฟังก์ชัน `for`  ที่ใ

In [278]:
from IPython.display import display, Markdown

In [279]:
display(Markdown(response))

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  sum = 0
  for number in numbers:
    sum += number
  return sum

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1. **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะใช้ในการเรียกใช้งานโค้ด
2. **`sum = 0`**:  สร้างค่าคงสองตัว (`sum`)  เพื่อเก็บผลรวม
3. **`for number in numbers:`**:  วนซ้ำผ่านทุกตัวเลขใน `numbers`  (แต่ละตัวเลขจะถูกนำไปเก็บเป็นค่าใน `sum`)
4. **`sum += number`**:  เพิ่มค่า `number` เข้าไปใน `sum`  (ค่าที่เก็บผลรวม)
5. **`return sum`**:  คืนค่า `sum`  (ผลรวมของตัวเลขทั้งหมด)
6. **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวอย่าง `numbers`  ซึ่งมี 10 ตัวเลข
7. **`result = find_sum(numbers)`**:  เรียกใช้ `find_sum`  เพื่อหาผลรวมของ `numbers`
8. **`print(result)`**:  แสดงผลลัพธ์ที่ได้จาก `find_sum`
**ข้อดีของโค้ดนี้:**

* **อ่านง่าย:**  โค้ดนี้ใช้ฟังก์ชัน `for`  ที่ใช้ในการวนซ้ำ  ทำให้โค้ดอ่านง่ายและเข้าใจง่าย
* **มีประสิทธิภาพ:**  โค้ดนี้ทำงานได้อย่างรวดเร็ว  เนื่องจากใช้ `for`  ในการวนซ้ำ
* **แยกความซับซ้อน:**  โค้ดนี้แยกความซับซ้อนของคำสั่ง  ( `for`  และการใช้ `sum` ) ออกเป็นส่วนๆ   ทำให้เข้าใจง่ายขึ้น
* **ใช้ `sum`  เป็นตัวแปร:**  `sum`  เป็นตัวแปร  ทำให้โค้ดสามารถปรับเปลี่ยนค่าของ `sum` ได้ง่าย

**ตัวอย่างการใช้งาน:**

* `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`
* `result = find_sum(numbers)`
* `print(result)`

**ข้อควรระวัง:**

* **ตัวแวน (Variable)  ไม่จำเป็นต้องมีตัวแวน:**  ในโค้ดนี้ตัวแวน `numbers`  ไม่ได้มีตัวแวน  (เช่น  `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  จะทำให้โค้ดทำงานได้  แต่ในกรณีนี้  ตัวแวน `numbers`  ไม่จำเป็นต้องมีตัวแวน  
* **การใช้ `sum`  เป็นตัวแปร:**  การใช้ `sum`  เป็นตัวแปร  อาจทำให้โค้ดอ่านง่ายขึ้น  ในบางกรณี  แต่ในกรณีนี้  การใช้ `sum`  เป็นตัวแปร  อาจทำให้โค้ดทำงานได้  ถ้าไม่จำเป็น  



In [280]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [281]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [282]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [283]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [284]:
parser_chain = StrOutputParser()

In [285]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [286]:
result = full_chain.invoke({"question":"2x + 5 = 15"})


In [287]:
result

'วิธิีทำจากผลลัทธ์ต่อไปนี้:\n\n* **การหาค่าของ x:** ผลลัทธ์ (π) เป็นตัวแทนของค่าที่ทำให้เกิดการเปลี่ยนแปลงในสมการต่อไปนี้\n* **การหาค่าของ y:** ค่าของ x จะถูกหาจากผลลัทธ์โดยใช้สูตร:\n   * `x = π * (10/10)`\n* **การหาค่าของ y:** ค่าของ y จะถูกหาจากค่าของ x โดยใช้สูตร:\n   * `y = x / π`\n\nดังนั้น, วิธิีทำจากผลลัทธ์ต่อไปนี้:\n\n**x = π * (10/10)**\n\n**ข้อควรระวัง:**\n\n* **ค่า π:** ค่า π เป็นค่าที่ใช้ในการหาผลลัทธ์ ซึ่งอาจแตกต่างกันไปในแต่ละระบบการคำนวณที่แตกต่างกัน\n* **ค่า x:** ค่า x เป็นค่าที่ใช้ในการหาค่าของ y ซึ่งอาจแตกต่างกันไปในแต่ละระบบการคำนวณที่แตกต่างกัน\n* **ความแม่นยำ:** การหาค่าของ x และ y อาจมีความแม่นยำน้อยกว่าความแม่นยำของค่า π\n'

In [288]:
import os
from langchain_openai import ChatOpenAI

In [289]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [290]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [291]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [292]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [293]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [294]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [295]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [296]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [297]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [298]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [299]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [300]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 79.2 ms, sys: 8.01 ms, total: 87.2 ms
Wall time: 6.69 s


In [301]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "กินไม่ไหว", "ขนมไทย", "ไข่อร่อย", "เห้ยแต่ราคา", "รับไม่ไหว", "ว", "ไม่ไปซ้ำ", "แพงเกิน", "เกินไป", "เลิกกินแล้วมีขนมหวานให้กินฟรี", "ขนมไทย")}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "กาแฟ",
    "ร้านวาวี",
    "starbuck",
    "แฟนๆ Starbuck",
    "กาแฟอร่อย",
    "กาแฟราคาไม่แพง",
    "กินบ่อย",
    "จุกได้",
    "รสชาติไม่ธรรมดา",
    "มวยรุ่นเดียวกะ Starbuck",
    "เมพพพพ",
    "ซากอารยธรรมลาเต้เมพๆ",
    "รสชาติ 1",
    "รสชาติ 2",
    "เมนูอื่นๆ",
    "วิวรอกาแฟ",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ฉีกน้ำตาลเทใส่แล้วเผลอคน",
    "ถ่ายรูปมาให้ดู",
    "55+",
    "ซากอารยธรรมลาเต้เมพๆ",
    "อร่อย

In [302]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [303]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    22 seconds ago            


In [304]:
llm3   = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [305]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [306]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 126 ms, sys: 9.73 ms, total: 135 ms
Wall time: 20.9 s


In [307]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะช่วยคุณวิเคราะห์และดึงคำสำคัญจากรีวิวนี้ออกมาเป็น JSON ตามขั้นตอนที่คุณต้องการ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงร้านอาหารขนาดใหญ่ที่มีจุดเริ่มต้นเป็นห้องน้ำ ราคาอาหารค่อนข้างแพง โดยเฉพาะพะแนงห่อไข่ที่ราคาสูงเกินไป แต่ทอดมันหัวปลีอร่อย และมีขนมหวานฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้านอาหาร (ใหญ่)
*   ราคาอาหาร (แพง)
*   อาหาร (ทอดมันหัวปลี, ลาบไข่ต้ม, พะแนงห่อไข่)
*   ขนมหวาน (ฟรี)

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

1.  ราคา (แพง)
2.  อาหาร (ทอดมันหัวปลี)
3.  ขนาด (ใหญ่)

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "อาหาร", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณมีคำถามเพิ่มเติมหรือต้องการให้ผมช่วยวิเคราะห์อะไรอีก บอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ขั้นตอนที่ 1: อ่านรี

In [308]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [309]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    44 seconds ago            


In [310]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [311]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 132 ms, sys: 11.2 ms, total: 143 ms
Wall time: 13.5 s


In [312]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้ และจัดทำเป็น JSON ดังนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้เป็นการรีวิวร้านอาหาร ซึ่งมีประเด็นหลักๆ เกี่ยวกับขนาดร้าน, ความแปลกของสิ่งอำนวยความสะดวก, ความอร่อยของอาหารบางเมนู, ราคาอาหาร, และการมีของหวานฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้าน
*   ความแปลกของสิ่งอำนวยความสะดวก (ห้องน้ำ)
*   ความอร่อยของอาหาร (ทอดมันหัวปลี, พะแนงห่อไข่)
*   ราคาอาหาร
*   ของหวานฟรี

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อที่กล่าวมา ผมเลือกคำเหล่านี้เป็น keywords ที่สำคัญที่สุด:

1.  ราคา (เนื่องจากมีการกล่าวถึงราคาอาหารหลายครั้ง และเป็นประเด็นสำคัญที่ทำให้ผู้รีวิวไม่ประทับใจ)
2.  อาหาร (ครอบคลุมถึงเมนูต่างๆ เช่น ทอดมันหัวปลี, พะแนงห่อไข่)
3.  ขนาด (เนื่องจากมีการกล่าวถึงขนาดร้านที่ใหญ่มาก)

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "อาหาร", "ขนาด"]

In [313]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [314]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [315]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [316]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 130 ms, sys: 12.3 ms, total: 142 ms
Wall time: 11.9 s


In [317]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาดร้าน']

In [318]:
results[1]['text']['keywords']

['วาวี', 'กาแฟ', 'บรรยากาศ']

In [319]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [320]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 141 ms, sys: 11.3 ms, total: 153 ms
Wall time: 11.8 s


In [321]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคาแพง', 'รสชาติ', 'ขนาดร้าน'], ['วาวี', 'ลาเต้', 'บรรยากาศ'])

In [322]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [323]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [324]:
inputs = [{"review": r} for r in sample_reviews]

In [325]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 26.6 ms, sys: 1.43 ms, total: 28.1 ms
Wall time: 2.84 s


In [326]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [327]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [328]:
from tqdm import tqdm

In [329]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:29<00:00,  9.36s/it]


In [330]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความอุ่น
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คลาสสิก
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
Shibuya Honey Toast, ฮันนี่ โทสท์, ราคา
คาเฟ่, วานิลลา, คาโบนาร่า


In [331]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [332]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [333]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [334]:
context_extract = context_prompt | llm3 | context_parser

In [335]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [336]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [337]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 190347.36it/s]


In [338]:
len(context_inputs)

3000

In [339]:
context_inputs[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'keyword': 'ราคา'}

In [340]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:12:46<00:00,  4.37s/it]


In [341]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [342]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 232797.03it/s]


In [343]:
len(sentiment_inputs)

3000

In [344]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [345]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [43:07<00:00,  2.59s/it]


In [346]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [347]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [349]:
df.to_csv('results1.csv', index=False, encoding='utf-8-sig')